# Impoting all ther required libraries and modules

---

In [ ]:
import pandas as pd
import numpy as np


import nltk

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import string


# Extracting all the relevent features
---

In [2]:
train_data = pd.read_csv("Consumer_Train.csv")
test_data = pd.read_csv("Consumer_Test.csv")

train_data.drop(['Date received','Sub-product','Company public response','Company','State','ZIP code','Tags','Consumer consent provided?','Submitted via','Company response to consumer','Timely response?','Complaint ID','Consumer disputed?','Date sent to company'],axis = 1,inplace =True)
test_data.drop(['Date received','Sub-product','Company public response','Company','State','ZIP code','Tags','Consumer consent provided?','Submitted via','Company response to consumer','Timely response?','Complaint ID','Consumer disputed?','Date sent to company'],axis = 1,inplace =True)

In [3]:
train_data.isnull().sum()
test_data.isnull().sum()

Product                            0
Issue                              0
Sub-issue                       1996
Consumer complaint narrative       0
dtype: int64

# Converting the y_labels to a numerical datatype and transforming the features to vectors

---

In [4]:
#deleting all the null values
train_data.dropna(inplace =True)
test_data.dropna(inplace =True)

text_to_num = {}
def convert_to_num(val):
    return text_to_num[val]

uniq_elements = set(train_data['Product'])
x=1
for uniq_value in uniq_elements:
    if uniq_value not in text_to_num  and uniq_value != -1:
        text_to_num[uniq_value] = x
        x+=1

y_train = list(map(convert_to_num, train_data['Product']))
y_train = np.array(y_train)


Consumer_complaint_narrative_train = pd.Series(list(train_data['Consumer complaint narrative']),index=list(range(len(train_data))))
Consumer_complaint_narrative_test = pd.Series(list(test_data['Consumer complaint narrative']),index=list(range(len(test_data))))

X_train,X_test,y_train,y_test = train_test_split(Consumer_complaint_narrative_train,y_train, test_size = 0.30,shuffle = True ,random_state = 1234)

class preprocessor:
    def __init__(self, sentences):
        self.sen = sentences

    def sent_tokenizer(self,sent):

        sent = nltk.sent_tokenize(sent)

        for i in range(len(sent)):

            sent[i] = sent[i][:len(sent[i])-1]
        sent = ' '.join(sent)

        return sent

    def tokenize_it(self, sen):
        # instantiate tokenizer class
        sen_new = nltk.word_tokenize(sen)
        return sen_new

    def remove_stopW_and_punc(self, sen_tokens):
        stopwords_english = stopwords.words('english')
        sen_clean = []

        for word in sen_tokens: # Go through every word in your tokens list
            if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
                sen_clean.append(word)
        return sen_clean


    def sen_stemmer(self, sen_clean):
        # Instantiate stemming class
        stemmer = PorterStemmer()

        # Create an empty list to store the stems
        sen_stem = []

        for word in sen_clean:
            stem_word = stemmer.stem(word)  # stemming word
            sen_stem.append(stem_word)  # append to the list

        return sen_stem

      # Now we setup a controller which initiates each mehtod of the above declared
      # class in the correct chronological order.

    def controller(self, sen):
        temp_sen = self.sent_tokenizer(sen)
        temp_sen = self.tokenize_it(temp_sen)
        temp_sen = self.remove_stopW_and_punc(temp_sen)
        temp_sen = self.sen_stemmer(temp_sen)

        return temp_sen

    def run(self):
        processed_sens = []
        while self.sen:
            sen = self.sen.pop(0)
            processed_sen = self.controller(sen)
            processed_sens.append(processed_sen)

        for i in range(len(processed_sens)):
            processed_sens[i] = ' '.join(processed_sens[i])

        return processed_sens

nltk.download('stopwords')
nltk.download('punkt')
processed_sentences_train = preprocessor(list(np.array(X_train))).run()
# vectorization


from sklearn.feature_extraction.text import TfidfVectorizer

Vectorizer = TfidfVectorizer(use_idf = True)
Consumer_complaint_narrative_train = Vectorizer.fit_transform(processed_sentences_train)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhijit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abhijit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Converting the test dataset to vectors
---

In [5]:
text_to_num = {}
def convert_to_num(val):
    return text_to_num[val]

uniq_elements = set(test_data['Product'])
x=1
for uniq_value in uniq_elements:
    if uniq_value not in text_to_num  and uniq_value != -1:
        text_to_num[uniq_value] = x
        x+=1

y_test = list(map(convert_to_num, test_data['Product']))
y_test = np.array(y_test)


processed_sentences_test = preprocessor(list(np.array(Consumer_complaint_narrative_test))).run()
Consumer_complaint_narrative_test = Vectorizer.transform(processed_sentences_test)

# Defining the logistic regressor model class, fitting it on the training the data and predicting on the test data

---

In [ ]:
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(Consumer_complaint_narrative_train.toarray().tolist(),y_train)

y_pred_LR = model.predict(Consumer_complaint_narrative_test.toarray().tolist())

# Gauging the model
---

In [7]:
print("The F1 score of the Logistic model is:", f1_score(y_test,y_pred_LR,average='weighted'))

The F1 score of the Logistic model is: 0.8928997297684136


### The f1_score is 89% as I didn't have the required RAM to train whole of the data. I have only used 70% of the data to train which is why the f1_score is not that good.